In [14]:
import itertools as it
import pandas as pd 
import numpy as np

In [ ]:
class RMvisualiser():


    #VALUES CAN CONTAIN NAN VALUE AND SHOULD BE FILTERED

    #load the data
    def load_data(self,path):
        self.data = pd.read_excel(path,usecols="G:AG")


    #return all cells
    def get_full(self):
        return self.data.to_numpy()


    #return all IL values
    def get_all(self):
        return self.data.to_numpy()[3:,1:]
    

    #get all IL values for a specific wavelengths
    def get_all_wavelength(self, wavelength):
        
        temp = self.data.to_numpy()[3:,0] == wavelength

        return self.data.to_numpy()[3:,0:][temp][:,1:]


    # get all IL values for a specific connector index 
    def get_connector(self,index):

        return self.get_all()[:,index]


    # get all IL values for a specific fiber(jumper) index 
    def get_fiber(self,index):

        return self.get_all()[:,[2*index,2*index+1]]
    
    # filter the data based on a list of fibers(jumpers to include)
    # example :

    # select fibers 1,2,3,4 
    # RM.select_n_fibers([0,1,2,3])

    def select_n_fibers(self,indecies):


        wave_adjusted_indecies = []
        adjusted_indecies = []
        for i in indecies:
            wave_adjusted_indecies.append(6*i)
            wave_adjusted_indecies.append(6*i+1)
            wave_adjusted_indecies.append(6*i+2)
            wave_adjusted_indecies.append(6*i+3)
            wave_adjusted_indecies.append(6*i+4)
            wave_adjusted_indecies.append(6*i+5)

            adjusted_indecies.append(2*i)
            adjusted_indecies.append(2*i+1)


        temp = self.get_all()[wave_adjusted_indecies][:,adjusted_indecies]

        return temp
    

    #return a list with elements containing data for all
    #possible combinations of 10 fibers chosen from 13 

    def get_all_10connectors(self):

        all_combinations_tupples = it.combinations(range(0,13),10)

        full_data = []

        for combination in all_combinations_tupples:

            data = self.select_n_fibers(list(combination)).flatten()

            data = data.astype(float)
            data = data[~np.isnan(data)]

            full_data.append(data)
        
        return full_data
    

    #returns a mean(avg) value of IL for
    # possible combinations of connectors
    def get_mean_all_10connectors(self):


        return list(map(lambda x : x.mean(),self.get_all_10connectors()))
    
    
    #returns a 97th percentile value of IL for
    # possible combinations of connectors
    def get_97percentiles(self):

        full_data = self.get_all_10connectors()

        return [np.percentile(x,97) for x in full_data]


    #returns the average value of IL for all connectors

    def average_connectors(self):

        ILs = []
        for i in range(26):
            IL = self.get_connector(i).flatten()
            IL = IL.astype(float)
            IL = IL[~np.isnan(IL)]

            ILs.append(IL.mean())

        return ILs
    
    #returns the average value of IL for all filbers(jumpers)

    def average_fibers(self):
        ILs = []
        for i in range(13):
            IL = self.get_fiber(i).flatten()
            IL = IL.astype(float)
            IL = IL[~np.isnan(IL)]

            ILs.append(IL.mean())

        return ILs
            

    #returns the standard deviation for all connectors

    def std_connectors(self):

        ILs = []
        for i in range(26):
            IL = self.get_connector(i).flatten()
            IL = IL.astype(float)
            IL = IL[~np.isnan(IL)]

            ILs.append(np.std(IL))

        return ILs
        
    # returns the ranking of connectors based on the average IL

    def connector_ranking(self):

        means = self.average_connectors()

        return np.argsort(means)
    

    # returns the mean values of ILs per connector 

    # ILs = RM.connector_wavelength_mean(1550)
    def connetor_wavelength_mean(self,wavelength):

        data = self.get_all_wavelength(wavelength)

        averages = []
        for i in range(26):
            IL = data[:,i].flatten()
            IL = IL.astype(float)
            IL = IL[~np.isnan(IL)]

            averages.append(IL.mean())


        return averages
    
    
    # returns the std of ILs per connector 

    # ILs = RM.connector_wavelength_mean(1550)
    def connetor_wavelength_ranking_std(self,wavelength):

        data = self.get_all_wavelength(wavelength)

        stds = []
        for i in range(26):
            IL = data[:,i].flatten()
            IL = IL.astype(float)
            IL = IL[~np.isnan(IL)]

            stds.append(np.std(IL))


        return stds
    

     # returns the mean values of ILs per jumper 

    # ILs = RM.jumper_wavelength_mean(1550)
    def jumper_wavelength_mean(self,wavelength):

        data = self.get_all_wavelength(wavelength)

        averages = []
        for i in range(13):
            IL = data[:,[2*i,2*i+1]].flatten()
            IL = IL.astype(float)
            IL = IL[~np.isnan(IL)]

            averages.append(IL.mean())


        return averages
    
    
    # returns the std of ILs per jumper

    # ILs = RM.jumper_wavelength_ranking_mean(1550)
    def jumper_wavelength_ranking_std(self,wavelength):

        data = self.get_all_wavelength(wavelength)

        stds = []
        for i in range(13):
            IL = data[:,[2*i,2*i+1]].flatten()
            IL = IL.astype(float)
            IL = IL[~np.isnan(IL)]

            stds.append(np.std(IL))


        return stds

In [ ]:
RM = RMvisualiser()

RM.load_data("RM example data.xlsx")


RM.get_all()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(RM.get_mean_all_10connectors(),20)
plt.show()

In [ ]:
import pandas as pd
import numpy as np

In [2]:
n_fibers = 12
n_connectors = 26
n_wavelengths = 3

In [ ]:
Sheet = np.zeros((n_connectors*n_connectors+2,n_fibers+2),dtype=object)

In [4]:
Sheet[0,0] = "Reference Configuration"
Sheet[1,0] = "Reference Connector"
Sheet[0,1] = "Fiber Number"
Sheet[1,1] = "DUT"

Sheet[1,2:n_fibers+2] = np.linspace(1,n_fibers,n_fibers)
Sheet[2:n_connectors*n_connectors+2,1] = np.tile(np.linspace(1,n_connectors,n_connectors),n_connectors)
Sheet[2:n_connectors*n_connectors+2,0] = np.repeat(np.linspace(1,n_connectors,n_connectors),n_connectors)



In [ ]:

df = pd.DataFrame(Sheet)

In [ ]:
import xlsxwriter

In [ ]:
writer = pd.ExcelWriter("test.xlsx", engine="xlsxwriter")
df.to_excel(writer,index = False,header=False)
workbook = writer.book

In [ ]:
merge_format = workbook.add_format(
    {
        "bold": 1,
        "border": 1,
        "align": "center",
        "valign": "vcenter",
    }
)

In [ ]:
for worksheet in workbook.worksheets():
    column_letter = xlsxwriter.utility.xl_col_to_name(n_fibers+2)

    print(f"C1:{column_letter}1")
    worksheet.set_column("A:B",20)
    worksheet.merge_range(f"C1:{column_letter}1", "Fiber Number",merge_format)
    for i in range(n_connectors):
        worksheet.merge_range(f"A{3+i*n_connectors}:A{2+(i+1)*n_connectors}", f"{i+1}",merge_format)

In [ ]:
writer.close()

In [1]:
import data_loading

dc = data_loading.DataCore()

In [2]:
dc.create_excel_template(14,n_fibers=4,n_wavelengths=3)

In [4]:
dc.load_excel("ExampleData.xlsx")

In [5]:
dc.excel_data

{1550.0:                            0             1             2         3         4  \
 0    Reference Configuration  Fiber Number  Fiber Number       NaN       NaN   
 1        Reference Connector           DUT             1  2.000000  3.000000   
 2                          1             1       0.18836  0.117686  0.104654   
 3                        NaN             2      0.141443  0.072320  0.043704   
 4                        NaN             3      0.080465  0.019479  0.082475   
 ..                       ...           ...           ...       ...       ...   
 193                      NaN            10       0.03019  0.003574  0.116242   
 194                      NaN            11      0.034772  0.127943  0.016793   
 195                      NaN            12      0.054064  0.097657  0.199341   
 196                      NaN            13      0.171099  0.148136  0.140722   
 197                      NaN            14      0.154672  0.060277  0.154652   
 
             5  
 

In [6]:
import regex

In [5]:
regex.findall(r"[-+]?\d*\.\d+|\d+nm","145nm")

NameError: name 'regex' is not defined

In [13]:
regex.match(r"[-+]?\d*\.\d+|\d+nm","145nm").

<regex.Match object; span=(0, 5), match='145nm'>

In [9]:
import pandas as pd
ef = pd.ExcelFile("template.xlsx")

In [10]:
ef.sheet_names

['wavelength_0', 'wavelength_1', 'wavelength_2']

In [11]:
df = ef.parse(ef.sheet_names[0])

In [12]:
df.head()

,Reference Configuration,Fiber Number,Fiber Number.1,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Reference Connector,DUT,1,2,3,4
1,1,1,0,0,0,0
2,NaN,2,0,0,0,0
3,NaN,3,0,0,0,0
4,NaN,4,0,0,0,0
